# Building Machine Learning Classifiers: Random Forest on a holdout test set

### Read in & clean text

In [21]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['body_text'])

x_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(x_tfidf.toarray())], axis=1)
x_features.head()

body_len  punct%    0    1    2    3    4    5    6    7  ...  8094  8095  \
0       128     4.7  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1        49     4.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2        62     3.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3        28     7.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4       135     4.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   

   8096  8097  8098  8099  8100  8101  8102  8103  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 8106 columns]

### Explore RandomForestClassifier through Holdout Set

In [22]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [23]:
#convert column names into strings:
column_numbers = x_features.columns
column_strings = [str(x) for x in column_numbers]
x_features.columns = column_strings

#str_col_nums = [str(x) for x in column_numbers]

#create your train_set and give 'x_features' as your feature set
#provide your training results in 'label'
#commonly, we separate 20% of our data to be used as holdout
x_train, x_test, y_train, y_test = train_test_split(x_features, data['label'], test_size = 0.2)

from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators = 50, max_depth = 20, n_jobs = -1)
rf_model = rf.fit(x_train, y_train)

In [24]:
#get the feature importances from the training set and get the corresponding column. Combine them with ZIP
#then, sort the most importand features. Use reverse to the get the most important features to the top
#check top 10

sorted(zip(rf_model.feature_importances_, x_train.columns), reverse = True)[0:10]

#the output below illustrates that the body length is the most important feature
#this can be changed between initiations as the random forest model includes some random sampling

[(0.0672609220693021, 'body_len'),
 (0.040031218755136926, '2031'),
 (0.03553659644911965, '1803'),
 (0.03522648707167086, '7350'),
 (0.0248121554534447, '4796'),
 (0.02319163375867135, '6746'),
 (0.021267615934637912, '5988'),
 (0.020076924414262568, '3134'),
 (0.018012277693906394, '6285'),
 (0.014595279464817501, '5078')]

In [25]:
#make a prediction based on the x_features.
y_pred = rf_model.predict(x_test)

#use score function with predictions and labels
#we have to say what the positive label is. In this example, it's 'spam'
#for simplicity, use 'binary' to calculare average. Further guidance is available in sikitlearn documentation
precision, recall, fscore, support = score(y_test, y_pred, pos_label = 'spam', average = 'binary')



In [29]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred), 3)))

#the output below indicate that there was no false-spam
#0.387 of spams are put into inbox

Precision: 1.0 / Recall: 0.613 / Accuracy: 0.946
